## Visualize FFT on a periodic signal

**common harmonic sine wave:**<br><br>
${y(t)= \hat{y} ⋅ sin(\omega⋅t + \phi_0)}$$ or $${y(t)=\hat{y}⋅sin(2{\pi}f⋅t+φ_0)}$

$\hat{y} : amplitude$<br>
${\omega: angular frequency}$ in  ${1 \over s}$<br>
${f: frequency}$ in ${1 \over s}$;  period length Ts = ${1 \over f}$ in s<br>
${\phi}_0: phase$
<br><br>

**common rectangular wave:**
$ x(t) = rect(at) = \begin{cases}
    1       & \quad \text{for |t|} \leq {1 \over 2a} \\
    0       & \quad \text{otherwise }
  \end{cases}$
<br><br>
[wikipedia, rectangular functions](https://en.wikipedia.org/wiki/Rectangular_function)


**common triangular wave:**
$ x(t) = tri(t) = \begin{cases}
    1-\text{|t|}       & \quad \text{for |t| < 1} \\
    0       & \quad \text{otherwise}
  \end{cases}$
<br><br> 
[wikipedia, triangular function](https://en.wikipedia.org/wiki/Triangular_function)

In [1]:
# resources
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

#set backend for interactive toolbar
%matplotlib widget

In [2]:
#presets
MAX_Amplitude = 5
MAX_Frequenz = 1000
MAX_Phase = 180

fNoise = 780
timeBase = np.arange(0, 3, 1/10000)     # time base as vector for 0...3 s, sample rate:  1/fsample (default = 10kHz)

In [3]:
# define functions
#
def getSinus(amp,f,phase,time):
    return(amp*np.sin(2*np.pi*f*time + phase))

#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.square.html
def getRect(amp,f,time):
    return(amp*signal.square(2*np.pi*f*time))

#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.sawtooth.html#scipy.signal.sawtooth
def getTri(f,time):
    return(signal.sawtooth(2*np.pi*f*time, width=0.5))


def calc_FFT(fSample, signal):
    #https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fft.html#numpy.fft.fft
    # fft mit numpy.fft - routinens
    #
    # Calc DFT of given signal
    fftSig = np.fft.fft(signal)
    #
    # length of complex fft-vector
    n = len(fftSig)
    # a range frequenzen 
    freq = np.fft.fftfreq(n, 1/fSample)
    #
    # select positiv area of complex spectrum
    # Shift the zero-frequency component to the center of the spectrum.
    # - amplitudes and frequencies
    Y1_shift = np.fft.fftshift(fftSig)
    F1_shift = np.fft.fftshift(freq)
    #
    # find index of zero point.
    #
    iZero = int(np.ceil(n/2.0))
    #
    # select amplitudes and frequencies from 0 ... n/2 
    Y1_pos = Y1_shift[iZero:-1]
    F1_pos = F1_shift[iZero:-1]
    #
    # normalize amplitude with (2* 1/n) and get real-part
    #
    reSpectrum = 2 * 1/n * np.abs(Y1_pos)
    return(reSpectrum , F1_pos)
#

In [4]:
#interactive controls
# 
# signal menu
#
signalForms = ['sinusodial', 'rectangular', 'triangular']
signal_dropdown = widgets.Dropdown(description='select waveform', options=signalForms, value='sinusodial',\
                                   style={'description_width': 'initial'})
#Sliders
amp_1 = widgets.IntSlider(min=0, max=MAX_Amplitude, value=1, description="$\hat{y}_1$:")
frq_1 = widgets.IntSlider(min=0, max=MAX_Frequenz, value=10, step=5, description="$f_1$: in Hz")
phase_1 = widgets.IntSlider(min=0, max=MAX_Phase, value=0, description='$\phi_1$: in °')
#
amp_2 = widgets.IntSlider(min=0, max=MAX_Amplitude, value=1, description="$\hat{y}_2$:")
frq_2 = widgets.IntSlider(min=0, max=MAX_Frequenz, value=10, step=5, description="$f_2$: in Hz")
phase_2 = widgets.IntSlider(min=0, max=MAX_Phase, value=0, description='$\phi_2$: in °')
#
fSample = widgets.IntSlider(min=100, max=100000, value=10000, step=10, description='sample frequency',\
                            style={'description_width': 'initial'})
#
#checkbox for noise overlay
chkNoise = widgets.Checkbox(description='add noise signal')
#
# reset button um auf default-Werte zurückzu setzen
resetBtn = widgets.Button(description="reset",tooltip="set default values")

In [5]:
# Visualization
# draw diagrams
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
#
# 1. Define callback function to update the figures when a control element has changed
def reset_controls(btn):
    # Reset Button:
    fSample.value = 10000
    signal_dropdown.value = 'sinusodial'
    frq_1.value = 10
    frq_2.value = 10
    amp_1.value = 1
    amp_2.value = 1
    phase_1.value = 1
    phase_2.value = 1
    chkNoise.value = False
    update_view()


def update_view(*args):
    
    # update time base vector
    timeBase = np.arange(0, 3, 1/fSample.value)    # time base as vector for 0...3 s, sample rate:  1/fsample (default = 10kHz)
    
    #
    #Sampling frequency must be at least twice as high as the highest occurring frequency
    #(Nyquist - Theorem)
    # if not: IndexError while calculating FFT
    #
    if ((frq_1.value > 50) | (frq_2.value > 50)):
        fSample.min = 2* np.max([frq_1.value, frq_2.value])          

    if (chkNoise.value == True):        
        fSample.min = 2* np.max([frq_1.value, frq_2.value, fNoise])

    # dropdown controls
    #
    signaltype = signal_dropdown.value
    
    #-----------------------------------------------------------
    # compute original signal according selection
    #
    if (signaltype == "sinusodial"):
        # activate posibbly disabled controls 
        amp_2.disabled = False
        frq_2.disabled = False
        phase_2.disabled = False
        chkNoise.disabled = False
        phase_1.disabled = False

        ySin1 = getSinus(amp_1.value, frq_1.value, phase_1.value, timeBase)
        ySin2 = getSinus(amp_2.value, frq_2.value, phase_2.value, timeBase)
        ySum = ySin1 + ySin2
        if (chkNoise.value == True): # noise signal with a fixed frequency of 780Hz and a fixed amplitude of 5
            ySum = ySin1 + ySin2 + (5*np.sin(2*np.pi*780*timeBase))
    
    if (signaltype == "rectangular"):
        yRect1 = getRect(amp_1.value, frq_1.value, timeBase)
        # disable controls of 2. sinusodial signal
        #
        amp_2.disabled = True
        frq_2.disabled = True
        phase_2.disabled = True
        chkNoise.disabled = True
        phase_1.disabled = True
        #
        ySum = yRect1
        #
    if (signaltype == "triangular"):
        yTri = getTri(frq_1.value, timeBase)
        # disable controls of 2. sinusodial signal
        #
        amp_2.disabled = True
        frq_2.disabled = True
        phase_2.disabled = True
        chkNoise.disabled = True
        phase_1.disabled = True
        #
        ySum = yTri
        
    #-----------------------------------------------------------
    # compute FFT
    #
    # amplitude spectrum of original-Signals:
    #
    fftSig, frqSig = calc_FFT(fSample.value, ySum)
    #
    # retransformation
    # (use complete complex DFT here)
    #
#     reOrig = np.fft.ifft(np.fft.fft(ySum))
    
    #-----------------------------------------------------------
    # update figures
    #
    # limit time base to 3 periods:
    if (frq_1.value < 2):
        xEnd = 3
    if (frq_1.value > 1):
        xEnd = 3* 1/frq_1.value
    if (signaltype == 'sinusodial'): 
        if ((frq_1.value > 0) & (frq_2.value > 0) & (frq_1.value > frq_2.value)):
            xEnd = 3* 1/frq_2.value
    #
    # limit x-range of FFT-diagram:
    fftError = True
    xFFTEnd = len(frqSig)
    add_f = 0
    peaks,_ = signal.find_peaks(fftSig, height=(0.01, 10))
    if (np.count_nonzero(peaks) > 0):
        fftError = False
        xFFTEnd = np.max(peaks)
        add_f = (len(frqSig) - xFFTEnd) -1
        if add_f > 100:
            add_f = 100

    # fig_1: original-signal in time domain
    #
    axes[0].clear()
    axes[0].set_title("signal in time domain")
    axes[0].plot(timeBase, ySum, linestyle='-', color='r', label=r"$y(t)_{{{}}}$".format(signaltype))
    if (signaltype == "sinusodial"):
        axes[0].plot(timeBase,ySin1, linewidth = 1, linestyle='--', color = 'b', label='$y_{1(t)}$')
        axes[0].plot(timeBase,ySin2, linewidth = 1, linestyle='dotted', color = '0.5', label='$y_{2(t)}$')
    axes[0].set_xlabel("time /s")
    axes[0].set_ylabel("amplitude")
    axes[0].set_xlim(timeBase[0], xEnd)
    axes[0].legend(loc="best")
    axes[0].grid(True)
    
    # fig_2: fft of original-signals
    #
    axes[1].clear()
    if (signaltype == "sinusodial"):
        axes[1].set_title("FFT of sum signal")
    else:
        axes[1].set_title("FFT of left signal")
    axes[1].plot(frqSig, fftSig, color='C1')
    axes[1].set_xlabel("f /Hz")
    axes[1].set_ylabel("amplitude")
    if fftError != True:
        axes[1].set_xlim(frqSig[0], frqSig[xFFTEnd+add_f])
    else:
        yLim = plt.ylim()
        yPos = yLim[0] + yLim[-1] / 2
        bbox_props = dict(boxstyle="round", fc=(1, 0.4, 0.4), ec="0.5", alpha=0.9)
        axes[1].text(frqSig[int(np.ceil(fSample.value/2))], yPos, "please check sample frequency",\
                     ha="center", va="center", size=10, bbox=bbox_props)
    
    axes[1].grid(True)
    
    fig.tight_layout()

#--------------------------------------------------------
# 2. Assign the callback function with the 'observe' function to the controls
signal_dropdown.observe(update_view, 'value')
fSample.observe(update_view,'value')
amp_1.observe(update_view,'value')
amp_2.observe(update_view,'value')
frq_1.observe(update_view,'value')
frq_2.observe(update_view,'value')
phase_1.observe(update_view,'value')
phase_2.observe(update_view,'value')
chkNoise.observe(update_view,'value')
resetBtn.on_click(reset_controls)


#--------------------------------------------------------
# run app
#
# force figure drawing
update_view()
#
#
# use 'widgets.VBox / .HBox' to arange the controls
#
# box-layout definieren
box_layout = widgets.Layout(display='inline-flex',flex_flow='column',align_items='flex-start',width='90%')

widgets.VBox([widgets.HBox([signal_dropdown, chkNoise]), fSample, widgets.HBox([amp_1, amp_2]),\
                            widgets.HBox([frq_1, frq_2]),widgets.HBox([phase_1, phase_2]),\
             resetBtn],\
            layout = box_layout)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Copyright © 2020 IUBH Internationale Hochschule